# HW1: Frame-Level Speech Recognition

In this homework, you will be working with MFCC data consisting of 27 features at each time step/frame. Your model should be able to recognize the phoneme occured in that frame.

# Libraries

In [1]:
import torch
import numpy as np
from torchsummaryX import summary
import sklearn
import gc
import zipfile
import pandas as pd

from tqdm.auto import tqdm
import os
import datetime
import wandb
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Device: ", device)

Device:  cuda


In [2]:
### PHONEME LIST
PHONEMES = [
            '[SIL]',   'AA',    'AE',    'AH',    'AO',    'AW',    'AY',
            'B',     'CH',    'D',     'DH',    'EH',    'ER',    'EY',
            'F',     'G',     'HH',    'IH',    'IY',    'JH',    'K',
            'L',     'M',     'N',     'NG',    'OW',    'OY',    'P',
            'R',     'S',     'SH',    'T',     'TH',    'UH',    'UW',
            'V',     'W',     'Y',     'Z',     'ZH',    '[SOS]', '[EOS]']

# Kaggle

This section contains code that helps you install kaggle's API, creating kaggle.json with you username and API key details. Make sure to input those in the given code to ensure you can download data from the competition successfully.

In [3]:
with open("/home/idoit/.kaggle/kaggle.json", "w+") as f:
    f.write('{"username":"elainejiang6","key":"524b60f24f581c320573ebba88cedbbf"}')
    # Put your kaggle username & key here

!chmod 600 /home/idoit/.kaggle/kaggle.json

In [4]:
# commands to download data from kaggle
# !kaggle competitions download -c 11785-hw1p2-s24
# !unzip -qo 11785-hw1p2-s24.zip -d ''

# 



Dataset

This section covers the dataset/dataloader class for speech data. You will have to spend time writing code to create this class successfully. We have given you a lot of comments guiding you on what code to write at each stage, from top to bottom of the class. Please try and take your time figuring this out, as it will immensely help in creating dataset/dataloader classes for future homeworks.

Before running the following cells, please take some time to analyse the structure of data. Try loading a single MFCC and its transcipt, print out the shapes and print out the values. Do the transcripts look like phonemes?

In [5]:
# Dataset class to load train and validation data

class AudioDataset(torch.utils.data.Dataset):

    def __init__(self, root, phonemes = PHONEMES, context=0, partition= "train-clean-100"): # Feel free to add more arguments

        self.context    = context
        self.phonemes   = phonemes

        self.mfcc_dir       = os.path.join(root, partition, 'mfcc')
        self.transcript_dir = os.path.join(root, partition, 'transcript')

        mfcc_names          = sorted(os.listdir(self.mfcc_dir))
        transcript_names    = sorted(os.listdir(self.transcript_dir))

        # Making sure that we have the same no. of mfcc and transcripts
        assert len(mfcc_names) == len(transcript_names)

        self.mfccs, self.transcripts = [], []

        # TODO: Iterate through mfccs and transcripts
        for i in range(len(mfcc_names)):
        #   Load a single mfcc
            mfcc_path = os.path.join(self.mfcc_dir, mfcc_names[i])
            mfcc        = np.load(mfcc_path)
        #   Do Cepstral Normalization of mfcc (explained in writeup)
            mfcc -= np.mean(mfcc, axis=0)
            mfcc = mfcc / (np.std(mfcc, axis=0) + 1e-5)
        #   Load the corresponding transcript
            # Remove [SOS] and [EOS] from the transcript
            # (Is there an efficient way to do this without traversing through the transcript?)
            # Note that SOS will always be in the starting and EOS at end, as the name suggests.

            transcript_path = os.path.join(self.transcript_dir, transcript_names[i])
            transcript = np.load(transcript_path, allow_pickle=True)

            # Remove [SOS] and [EOS] from the transcript
            if transcript[0] == '[SOS]' and transcript[-1] == '[EOS]':
              transcript = transcript[1:-1]
            else:
              transcript = transcript

        #   Append each mfcc to self.mfcc, transcript to self.transcript
            self.mfccs.append(mfcc)
            self.transcripts.append(transcript)

        # NOTE:
        # Each mfcc is of shape T1 x 27, T2 x 27, ...
        # Each transcript is of shape (T1+2) x 27, (T2+2) x 27 before removing [SOS] and [EOS]

        # TODO: Concatenate all mfccs in self.mfccs such that
        # the final shape is T x 27 (Where T = T1 + T2 + ...)
        self.mfccs          = np.concatenate(self.mfccs, axis=0)

        # TODO: Concatenate all transcripts in self.transcripts such that
        # the final shape is (T,) meaning, each time step has one phoneme output
        self.transcripts    = np.concatenate(self.transcripts, axis=0)
        # Hint: Use numpy to concatenate

        # Length of the dataset is now the length of concatenated mfccs/transcripts
        self.length = len(self.mfccs)

        # Take some time to think about what we have done.
        # self.mfcc is an array of the format (Frames x Features).
        # Our goal is to recognize phonemes of each frame
        # From hw0, you will be knowing what context is.
        # We can introduce context by padding zeros on top and bottom of self.mfcc
        if context > 0:
            pad_width = ((context, context), (0, 0))  # pad `context` frames at the start and end, no padding for features
            self.mfccs = np.pad(self.mfccs, pad_width=pad_width, mode='constant', constant_values=0)

        # The available phonemes in the transcript are of string data type
        # But the neural network cannot predict strings as such.
        # Hence, we map these phonemes to integers

        # TODO: Map the phonemes to their corresponding list indexes in self.phonemes
        phoneme_to_index = {phoneme: index for index, phoneme in enumerate(self.phonemes)}
        self.transcripts = np.vectorize(phoneme_to_index.get)(self.transcripts)
        # Now, if an element in self.transcript is 0, it means that it is 'SIL' (as per the above example)

    def __len__(self):
        return self.length

    def __getitem__(self, ind):

        # TODO: Based on context and offset, return a frame at given index with context frames to the left, and right.
        context = self.context
        start_index = ind
        end_index = ind + 2 * context + 1
        frames = self.mfccs[start_index:end_index]

        # After slicing, you get an array of shape 2*context+1 x 27. But our MLP needs 1d data and not 2d.
        # frames = frames.flatten() # TODO: Flatten to get 1d data

        frames      = torch.from_numpy(frames).to(torch.float32) # Convert to tensors
        phonemes    = torch.tensor(self.transcripts[ind]).to(torch.long)

        return frames, phonemes

In [6]:
class AudioTestDataset(torch.utils.data.Dataset):

    # TODO: Create a test dataset class similar to the previous class but you dont have transcripts for this
    # Imp: Read the mfccs in sorted order, do NOT shuffle the data here or in your dataloader.
    def __init__(self, root, phonemes = PHONEMES, context=0, partition= "test-clean"):

        self.context    = context
        self.phonemes   = phonemes

        self.mfcc_dir       = os.path.join(root, partition, 'mfcc')
        mfcc_names          = sorted(os.listdir(self.mfcc_dir))


        self.mfccs = []

        for i in range(len(mfcc_names)):
            mfcc_path = os.path.join(self.mfcc_dir, mfcc_names[i])
            mfcc        = np.load(mfcc_path)
        #   Do Cepstral Normalization of mfcc (explained in writeup)
            mfcc -= np.mean(mfcc, axis=0)
            mfcc /= np.std(mfcc, axis=0) + 1e-5

            self.mfccs.append(mfcc)

        self.mfccs          = np.concatenate(self.mfccs, axis=0)
        self.length = len(self.mfccs)

        if context > 0:
            pad_width = ((context, context), (0, 0))  # pad `context` frames at the start and end, no padding for features
            self.mfccs = np.pad(self.mfccs, pad_width=pad_width, mode='constant', constant_values=0)

    def __len__(self):
        return self.length

    def __getitem__(self, ind):

        # TODO: Based on context and offset, return a frame at given index with context frames to the left, and right.
        context = self.context
        start_index = ind
        end_index = ind + 2 * context + 1
        frames = self.mfccs[start_index:end_index]

        # After slicing, you get an array of shape 2*context+1 x 27. But our MLP needs 1d data and not 2d.
        frames = frames.flatten() # TODO: Flatten to get 1d data

        frames      = torch.FloatTensor(frames) # Convert to tensors

        return frames

# Parameters Configuration

Storing your parameters and hyperparameters in a single configuration dictionary makes it easier to keep track of them during each experiment. It can also be used with weights and biases to log your parameters for each experiment and keep track of them across multiple experiments.

In [7]:
config = {
    'epochs'        : 20,
    'batch_size'    : 2048,
    'context'       : 20,
    'init_lr'       : 1e-4,
    'architecture'  : 'very-low-cutoff',
    'patience'      : 1,
    # Add more as you need them - e.g dropout values, weight decay, scheduler parameters
}

# Create Datasets

In [8]:
np.load('11-785-s24-hw1p2/train-clean-100/transcript/103-1240-0000.npy')

array(['[SOS]', '[SIL]', '[SIL]', ..., '[SIL]', '[SIL]', '[EOS]'],
      dtype='<U5')

In [9]:
#TODO: Create a dataset object using the AudioDataset class for the training data
train_data = AudioDataset(
    root = '11-785-s24-hw1p2',
    context=config['context']
    )

test_data = AudioTestDataset(root = '11-785-s24-hw1p2',
    context=config['context'])

val_data = AudioDataset(root = '11-785-s24-hw1p2',
    context=config['context'],partition= "dev-clean")

In [10]:
# Define dataloaders for train, val and test datasets
# Dataloaders will yield a batch of frames and phonemes of given batch_size at every iteration
# We shuffle train dataloader but not val & test dataloader. Why?

train_loader = torch.utils.data.DataLoader(
    dataset     = train_data,
    num_workers = 4,
    batch_size  = config['batch_size'],
    pin_memory  = True,
    shuffle     = True
)

val_loader = torch.utils.data.DataLoader(
    dataset     = val_data,
    num_workers = 2,
    batch_size  = config['batch_size'],
    pin_memory  = True,
    shuffle     = False
)

test_loader = torch.utils.data.DataLoader(
    dataset     = test_data,
    num_workers = 2,
    batch_size  = config['batch_size'],
    pin_memory  = True,
    shuffle     = False
)


print("Batch size     : ", config['batch_size'])
print("Context        : ", config['context'])
print("Input size     : ", (2*config['context']+1)*27)
print("Output symbols : ", len(PHONEMES))

print("Train dataset samples = {}, batches = {}".format(train_data.__len__(), len(train_loader)))
print("Validation dataset samples = {}, batches = {}".format(val_data.__len__(), len(val_loader)))
print("Test dataset samples = {}, batches = {}".format(test_data.__len__(), len(test_loader)))

Batch size     :  2048
Context        :  20
Input size     :  1107
Output symbols :  42
Train dataset samples = 36091157, batches = 17623
Validation dataset samples = 1928204, batches = 942
Test dataset samples = 1934138, batches = 945


In [11]:
# Testing code to check if your data loaders are working
for i, data in enumerate(train_loader):
    frames, phoneme = data
    print(frames.shape, phoneme.shape)
    break

torch.Size([2048, 41, 27]) torch.Size([2048])


# Network Architecture


This section defines your network architecture for the homework. We have given you a sample architecture that can easily clear the very low cutoff for the early submission deadline.

In [12]:
import torch.nn as nn
import math


class MLPBlock(nn.Module):
  def __init__(self, d_model, p, skip):

    super(MLPBlock, self).__init__()

    self.skip = skip
    self.model = torch.nn.Sequential(
      torch.nn.BatchNorm1d(d_model),
      torch.nn.Linear(d_model, d_model),
      torch.nn.GELU(),
      torch.nn.Dropout(p),
    )

  def forward(self, x):
      out = self.model(x)
      if self.skip:
        out = out + x
      return out

      

# This architecture will make you cross the very low cutoff
# However, you need to run a lot of experiments to cross the medium or high cutoff
class Network(torch.nn.Module):

    def __init__(self,
                 input_size,
                 output_size,
                 d_model= 1200,
                 num_block = 7,
                 p=0.3,
                 skip=True,
                ):
        super(Network, self).__init__()

        models = []
        models.append(nn.Flatten())
        models.append(nn.Linear(input_size, d_model))
        models.append(nn.GELU())
        for i in range(num_block):
          models.append(MLPBlock(d_model, p, skip))
        models.append(nn.Linear(d_model, output_size))

        self.model = nn.Sequential(*models)

    def forward(self, x):
        out = self.model(x)
        return out

# class Network(nn.Module):
#   def __init__(self, input_size, output_size):
#     super().__init__()

#     d_model = 256
#     n_tokens = 4

#     self.d_model = d_model
#     self.n_tokens = n_tokens

#     # input mapping
#     self.encode = nn.Sequential(
#       nn.LayerNorm(input_size),
#       nn.Linear(input_size, d_model * n_tokens),
#       nn.GELU(),
#     )

#     # transformer
#     encoder_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=4, activation='gelu', batch_first=True)
#     self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=1)

#     # classier
#     self.classier_head = nn.Linear(d_model, output_size)

#   def forward(self, x):
#     x = self.encode(x)
#     x = x.view(list(x.shape[:-1]) + [self.n_tokens, self.d_model])
#     x = self.transformer_encoder(x).mean(-2)
#     return self.classier_head(x)

# Network(INPUT_SIZE, len(train_data.phonemes))(torch.randn(1, 41, 27))


# class PositionalEncoding(nn.Module):

#     def __init__(self, d_model: int, dropout: float = 0.1, max_len: int = 5000):
#         super().__init__()
#         self.dropout = nn.Dropout(p=dropout)

#         position = torch.arange(max_len).unsqueeze(1)
#         div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
#         pe = torch.zeros(max_len, d_model)
#         pe[:, 0::2] = torch.sin(position * div_term)
#         pe[:, 1::2] = torch.cos(position * div_term)
#         self.register_buffer('pe', pe)

#     def forward(self, x):
#         """
#         Arguments:
#             x: Tensor, shape ``[batch_size, seq_len, embedding_dim]``
#         """
#         x = x + self.pe[:x.size(1)][None]
#         return self.dropout(x)

# class Network(nn.Module):
#   def __init__(self, input_size, output_size):
#     super().__init__()

#     d_model = 256
#     frame_d = 27

#     self.d_model = d_model
#     self.pe = PositionalEncoding(d_model)

#     # input mapping
#     self.encode = nn.Sequential(
#       nn.Linear(frame_d, d_model),
#       nn.GELU(),
#       nn.LayerNorm(d_model),
#     )

#     # transformer
#     encoder_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=4, activation='gelu', batch_first=True)
#     self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=1)

#     # classier
#     self.classier_head = nn.Sequential(
#       nn.Linear(d_model, d_model),
#       nn.GELU(),
#       nn.Linear(d_model, output_size)
#     )

#   def forward(self, x):
#     x = self.encode(x)
#     x = self.pe(x)
#     x = self.transformer_encoder(x)[:, x.size(1) // 2]
#     return self.classier_head(x)

In [13]:
class FocalLoss(nn.Module):
    def __init__(self, gamma=2.0, alpha=0.25):
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        self.alpha = alpha

    def forward(self, input, target):
        # compute cross entropy loss
        ce_loss = torch.nn.functional.cross_entropy(input, target, reduction='none')

        # compute focal loss
        pt = torch.exp(-ce_loss)
        focal_loss = self.alpha * (1 - pt)**self.gamma * ce_loss

        return focal_loss.mean()

In [14]:
# from torch.cuda.amp import GradScaler

# INPUT_SIZE  = (2*config['context'] + 1) * 27 # Why is this the case?
# model       = Network(INPUT_SIZE, len(train_data.phonemes)).to(device)
# model = torch.nn.DataParallel(model)

# # criterion = torch.nn.CrossEntropyLoss().to(device) # Defining Loss function.
# criterion = FocalLoss().to(device)
# # We use CE because the task is multi-class classification

# optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5) #Defining Optimizer

# scaler = GradScaler()

# model.train()
# tloss, tacc = 0, 0 # Monitoring loss and accuracy
# batch_bar   = tqdm(total=len(train_loader), dynamic_ncols=True, leave=False, position=0, desc='Train')

# for i, (frames, phonemes) in enumerate(train_loader):

#     ### Initialize Gradients
#     # optimizer.zero_grad()

#     ### Move Data to Device (Ideally GPU)
#     frames      = frames.to(device)
#     phonemes    = phonemes.to(device)

#     ### Forward Propagation
#     with torch.autocast(device_type='cuda', dtype=torch.float16):
#         logits  = model(frames)
#         loss    = criterion(logits, phonemes)

#     ### Backward Propagation
#     loss.backward()

#     ### Gradient Descent
#     optimizer.step()

#     tloss   += loss.item()
#     tacc    += torch.sum(torch.argmax(logits, dim= 1) == phonemes).item()/logits.shape[0]

#     batch_bar.set_postfix(loss="{:.04f}".format(float(tloss / (i + 1))),
#                           acc="{:.04f}%".format(float(tacc*100 / (i + 1))))
#     batch_bar.update()

#     ### Release memory
#     # del frames, phonemes, logits
#     # torch.cuda.empty_cache()

# Define Model, Loss Function and Optimizer

Here we define the model, loss function, optimizer and optionally a learning rate scheduler.

In [15]:
INPUT_SIZE  = (2*config['context'] + 1) * 27 # Why is this the case?
model       = Network(INPUT_SIZE, len(train_data.phonemes)).to(device)
# summary(model, frames.to(device))
model(frames.to(device)).shape
# Check number of parameters of your network
# Remember, you are limited to 24 million parameters for HW1 (including ensembles)

torch.Size([2048, 42])

In [16]:
from torch.optim.lr_scheduler import ReduceLROnPlateau,StepLR,MultiStepLR

In [17]:
criterion = torch.nn.CrossEntropyLoss() # Defining Loss function.
# We use CE because the task is multi-class classification

optimizer = torch.optim.AdamW(model.parameters(), lr= config['init_lr']) #Defining Optimizer
# scheduler = ReduceLROnPlateau(optimizer, 'min', patience = config['patience'])
scheduler = MultiStepLR(optimizer,  milestones=[3,6,9,12,15,18], gamma=0.7)
# Recommended : Define Scheduler for Learning Rate,
# including but not limited to StepLR, MultiStepLR, CosineAnnealingLR, ReduceLROnPlateau, etc.
# You can refer to Pytorch documentation for more information on how to use them.

# Is your training time very high?
# Look into mixed precision training if your GPU (Tesla T4, V100, etc) can make use of it
# Refer - https://pytorch.org/docs/stable/notes/amp_examples.html

# Training and Validation Functions

This section covers the training, and validation functions for each epoch of running your experiment with a given model architecture. The code has been provided to you, but we recommend going through the comments to understand the workflow to enable you to write these loops for future HWs.

In [18]:
torch.cuda.empty_cache()
gc.collect()

150

In [19]:
def train(model, dataloader, optimizer, criterion, epoch):

    model.train()
    tloss, tacc = 0, 0 # Monitoring loss and accuracy
    batch_bar   = tqdm(total=len(train_loader), dynamic_ncols=True, leave=False, position=0, desc='Train')
    st_ind = epoch * len(dataloader)

    for i, (frames, phonemes) in enumerate(dataloader):

        ### Initialize Gradients
        optimizer.zero_grad()

        ### Move Data to Device (Ideally GPU)
        frames      = frames.to(device)
        phonemes    = phonemes.to(device)

        ### Forward Propagation
        logits  = model(frames)

        ### Loss Calculation
        loss    = criterion(logits, phonemes)

        ### Backward Propagation
        loss.backward()

        ### Gradient Descent
        optimizer.step()

        tloss   += loss.item()
        tacc    += torch.sum(torch.argmax(logits, dim= 1) == phonemes).item()/logits.shape[0]

        batch_bar.set_postfix(loss="{:.04f}".format(float(tloss / (i + 1))),
                              acc="{:.04f}%".format(float(tacc*100 / (i + 1))))
        batch_bar.update()

        # log
        if i % 1000 == 0:
          wandb.log({
            'train/loss': float(tloss / (i + 1)),
            'train/acc': float(tacc*100 / (i + 1)),
            'step': st_ind + i,
          })

        ### Release memory
        # del frames, phonemes, logits
        # torch.cuda.empty_cache()

    batch_bar.close()
    tloss   /= len(train_loader)
    tacc    /= len(train_loader)

    return tloss, tacc

In [20]:
def eval(model, dataloader):

    model.eval() # set model in evaluation mode
    vloss, vacc = 0, 0 # Monitoring loss and accuracy
    batch_bar   = tqdm(total=len(val_loader), dynamic_ncols=True, position=0, leave=False, desc='Val')

    for i, (frames, phonemes) in enumerate(dataloader):

        ### Move data to device (ideally GPU)
        frames      = frames.to(device)
        phonemes    = phonemes.to(device)

        # makes sure that there are no gradients computed as we are not training the model now
        with torch.inference_mode():
            ### Forward Propagation
            logits  = model(frames)
            ### Loss Calculation
            loss    = criterion(logits, phonemes)

        vloss   += loss.item()
        vacc    += torch.sum(torch.argmax(logits, dim= 1) == phonemes).item()/logits.shape[0]

        # Do you think we need loss.backward() and optimizer.step() here?

        batch_bar.set_postfix(loss="{:.04f}".format(float(vloss / (i + 1))),
                              acc="{:.04f}%".format(float(vacc*100 / (i + 1))))
        batch_bar.update()

        ### Release memory
        del frames, phonemes, logits
        torch.cuda.empty_cache()

    batch_bar.close()
    vloss   /= len(val_loader)
    vacc    /= len(val_loader)

    return vloss, vacc

# Weights and Biases Setup

This section is to enable logging metrics and files with Weights and Biases. Please refer to wandb documentationa and recitation 0 that covers the use of weights and biases for logging, hyperparameter tuning and monitoring your runs for your homeworks. Using this tool makes it very easy to show results when submitting your code and models for homeworks, and also extremely useful for study groups to organize and run ablations under a single team in wandb.

We have written code for you to make use of it out of the box, so that you start using wandb for all your HWs from the beginning.

In [21]:
wandb.login(key="ae6f0b7581bd32fe30620ae74a71a7c2e4d30250") #API Key is in your wandb account, under settings (wandb.ai/settings)

wandb: Currently logged in as: jiangfan892. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/idoit/.netrc


True

In [22]:
# Create your wandb run
name = "run_sched_skip_steplr2"
run = wandb.init(
    name    = name, ### Wandb creates random run names if you skip this field, we recommend you give useful names
    reinit  = True, ### Allows reinitalizing runs when you re-run this cell
    #id     = "y28t31uz", ### Insert specific run id here if you want to resume a previous run
    #resume = "must", ### You need this to resume previous runs, but comment out reinit = True when using this
    project = "hw1p2", ### Project should be created in your wandb account
    config  = config ### Wandb Config for your run
)

In [23]:
### Save your model architecture as a string with str(model)
model_arch  = str(model)

### Save it in a txt file
arch_file   = open("model_arch.txt", "w")
file_write  = arch_file.write(model_arch)
arch_file.close()

### log it in your wandb run with wandb.save()
wandb.save('model_arch.txt')

['/home/idoit/Desktop/fanjiang/wandb/run-20240209_184826-pf1w6tfx/files/model_arch.txt']

# Experiment

Now, it is time to finally run your ablations! Have fun!

In [24]:
# Iterate over number of epochs to train and evaluate your model
torch.cuda.empty_cache()
gc.collect()
# wandb.watch(model, log="all")

for epoch in range(config['epochs']):

    print("\nEpoch {}/{}".format(epoch+1, config['epochs']))

    curr_lr                 = float(optimizer.param_groups[0]['lr'])
    train_loss, train_acc   = train(model, train_loader, optimizer, criterion, epoch)
    val_loss, val_acc       = eval(model, val_loader)
    scheduler.step()

    print("\tTrain Acc {:.04f}%\tTrain Loss {:.04f}\t Learning Rate {:.07f}".format(train_acc*100, train_loss, curr_lr))
    print("\tVal Acc {:.04f}%\tVal Loss {:.04f}".format(val_acc*100, val_loss))

    ### Log metrics at each epoch in your run
    # Optionally, you can log at each batch inside train/eval functions
    # (explore wandb documentation/wandb recitation)
    wandb.log({'train_acc': train_acc*100, 'train_loss': train_loss,
               'val_acc': val_acc*100, 'valid_loss': val_loss, 'lr': curr_lr, 'epoch': epoch,})

    ### Highly Recommended: Save checkpoint in drive and/or wandb if accuracy is better than your current best
    torch.save(model.state_dict(), F"{name}_model_{epoch}.pt")

### Finish your wandb run
run.finish()


Epoch 1/20


Train:   0%|                                                                            | 0/17623 [00:00<?, ?i…

Val:   0%|                                                                                | 0/942 [00:00<?, ?i…

	Train Acc 74.2924%	Train Loss 0.8005	 Learning Rate 0.0001000
	Val Acc 78.6675%	Val Loss 0.6473

Epoch 2/20


Train:   0%|                                                                            | 0/17623 [00:00<?, ?i…

...

	Train Acc 88.2962%	Train Loss 0.3313	 Learning Rate 0.0000118
	Val Acc 82.2021%	Val Loss 0.5687

Epoch 20/20


Train:   0%|                                                                            | 0/17623 [00:00<?, ?i…

Val:   0%|                                                                                | 0/942 [00:00<?, ?i…

	Train Acc 88.3589%	Train Loss 0.3294	 Learning Rate 0.0000118
	Val Acc 82.2289%	Val Loss 0.5696


epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
lr,███▆▆▆▄▄▄▃▃▃▂▂▂▁▁▁▁▁
step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/acc,▁▃▅▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇██████████████████████
train/loss,█▆▃▃▃▃▂▂▂▂▂▂▂▂▂▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_acc,▁▄▅▆▆▆▇▇▇▇▇▇████████
train_loss,█▅▄▃▃▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁
val_acc,▁▅▆▇▇▇██████████████
valid_loss,█▄▂▁▁▁▁▁▁▁▁▂▂▂▂▂▂▂▂▂
epoch,19
lr,1e-05


# Testing and submission to Kaggle

Before we get to the following code, make sure to see the format of submission given in *sample_submission.csv*. Once you have done so, it is time to fill the following function to complete your inference on test data. Refer the eval function from previous cells to get an idea of how to go about completing this function.

In [25]:
def test(model, test_loader):
    ### What you call for model to perform inference?
    model.eval # TODO train or eval?

    ### List to store predicted phonemes of test data
    test_predictions = []

    ### Which mode do you need to avoid gradients?
    with torch.no_grad(): # TODO

        for i, mfccs in enumerate(tqdm(test_loader)):

            mfccs   = mfccs.to(device)

            logits  = model(mfccs)

            ### Get most likely predicted phoneme with argmax
            predicted_phonemes_indices = logits.argmax(dim=1)

            ### How do you store predicted_phonemes with test_predictions? Hint, look at eval
            
            predicted_phonemes = [PHONEMES[index] for index in predicted_phonemes_indices.cpu().numpy()]
            test_predictions.extend(predicted_phonemes)


    return test_predictions

In [26]:
predictions = test(model, test_loader)

  0%|          | 0/945 [00:00<?, ?it/s]

In [27]:
### Create CSV file with predictions
with open("./submission9.csv", "w+") as f:
    f.write("id,label\n")
    for i in range(len(predictions)):
        f.write("{},{}\n".format(i, predictions[i]))

In [ ]:
### Submit to kaggle competition using kaggle API (Uncomment below to use)
# !kaggle competitions submit -c 11785-hw1p2-s24 -f ./submission.csv -m "Test Submission"

### However, its always safer to download the csv file and then upload to kaggle